# Title: csck700_ifc_parser

### The module parses an IFC model to extract the IFcElements, their properties and relationships and loads them to nodes and edges into Neo4J.

# 0. Table of content:

- [1. Settings:](#1.-Settings:)  
- [2. Imports:](#2.-Imports:)  
- [3. Helper functions:](#3.-Helper-functions:)  
    - [3.1. Proxy entities handling:](#3.1.-Proxy-entities-handling:)  
    - [3.2. IfcRelationships - Edges:](#3.2.-IfcRelationships---Edges:)  
    - [3.3. IfcEntities - Nodes:](#3.3.-IfcEntities---Nodes:)  
    - [3.4. Sanity Checks:](#3.3.-Sanity-Checks:)
    - [3.5. Graph schema definition:](#3.5.-Graph-schema-definition:)  
    - [3.6. Nodes insertion & Relationshsips/edges insertion:](#3.6.-Nodes-insertion-&-Relationshsips/edges-insertion:)
    - [3.7. Graph integrity:](#3.7.-Graph-integrity:)
- [4. Data load:](#4.-Data-load:)
- [5. Proxy entities handling:](#5.-Proxy-entities-handling:)  
- [6. IFC Parsing:](#5.-IFC-Parsing:)  
    - [6.1. IfcRelationships - Edges Preparation:](#6.1.-IfcRelationships---Edges-Preparation:)  
    - [6.2. IfcEntities - Nodes Preparation:](#6.2.-IfcEntities---Nodes-Preparation:)  
    - [6.3. Sanity Checks:](#6.3.-Sanity-Checks:)  
- [7. Neo4J graph:](#7.-Neo4J-graph:)
    - [7.1. Graph schema definition:](#7.1.-Graph-schema-definition:)
    - [7.2. Graph creation:](#7.2.-Graph-creation:)  
    - [7.3. Graph integrity:](#7.3.-Graph-integrity:)
    - [7.4. Exploration & visualization:](#7.4.-Exploration-&-visualization:)
    - [7.5. Graph summary:](#7.5.-Graph-summary:)      

# 1. Settings:

In [1]:
IFC_PATH = r"..\data\raw\Building-Structural.ifc"

NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "TitineTiteFiro@1952"
DATABASE = "capstone"

In [2]:
REL_MAP ={
    "IfcRelDefinesByProperties": ("RelatingPropertyDefinition", "RelatedObjects", None),
    "IfcRelDefinesByType": ("RelatingType", "RelatedObjects", "DEFINED_BY_TYPE"),
    "IfcRelAssociatesMaterial": ("RelatingMaterial", "RelatedObjects", "ASSOCIATED_MATERIAL"),
    "IfcRelContainedInSpatialStructure": ("RelatingStructure", "RelatedElements", "CONTAINED_IN"),
    "IfcRelAggregates": ("RelatingObject", "RelatedObjects", "AGGREGATES"),
    "IfcRelAssociatesClassification": ("RelatingClassification", "RelatedObjects", "ASSOCIATED_CLASSIFICATION")
} 

# 2. Imports:

In [3]:
import ifcopenshell
import ifcopenshell.util.element as util
from neo4j import GraphDatabase
from collections import Counter, defaultdict
# !pip install yfiles-jupyter-graphs-for-neo4j
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
import json
from IPython.display import JSON, display

# 3. Helper functions:

## 3.1. Proxy entities handling:

In [4]:
def proxies_analysis(model):
    """
    Identify proxy elements, their related relationships, and dependent entities
    that only exist through those relationships.
    """

    # proxies identification:
    proxies = {ent for ent in model if "Proxy" in ent.is_a()}

    # relationships involved in proxies:
    proxies_rels = {
        r
        for p in proxies
        for r in model.get_inverse(p)
        if hasattr(r, "is_a") and r.is_a("IfcRelationship")
    }

    # other-side entities, non-proxy, non-relationship:
    others = {
        e 
        for r in proxies_rels
        for v in r.get_info().values()
        for e in (v if isinstance(v, (list, tuple, set)) else [v])
        if hasattr(e, "is_a") and not e.is_a("IfcRelationship") and e not in proxies
    }

    # isolated: no other relationship beyond proxy_rels:
    isolated = {
        o for o in others
        if not any(r for r in model.get_inverse(o)
                  if hasattr(r, "is_a") and r.is_a("IfcRelationship") and r not in proxies_rels)
    }

    return proxies, proxies_rels, isolated

In [5]:
def proxies_drop(model):
    """
    Remove proxy elements, their associated relationships, and dependent isolated entities
    from the IFC model to produce a cleaned version.
    """

    proxies, proxies_rels, isolated = proxies_analysis(model)

    # remove relationships that touch proxies:
    for r in proxies_rels:
        # run("entity.remove", model, entity=r)
        model.remove(r)

    # remove ensuing isolated nodes:
    for i in isolated:
        # # run("entity.remove", model, entity=i)
        try:
            model.remove(e)
        except Exception:
            pass

    # remove proxies:
    for p in proxies:
        # run("entity.remove", model, entity=p)
        try:
            model.remove(p)
        except Exception:
            pass

    print("\n====PROXIES REMOVAL COUNTS:====\n")
    print(f"relationships: {len(proxies_rels)}")
    print(f"isolated nodes: {len(isolated)}")
    print(f"proxies: {len(proxies)}") 

    return model

## 3.2. IfcRelationships - Edges:

In [6]:
def identify_rel(model):
    """
    Identify all distinct IfcRelationship classes present in the model.
    """

    rel_classes = set()

    rels = model.by_type("IfcRelationship")

    for rel in rels:
        rel_classes.add(rel.is_a())

    print("Identified relationship classes:")  
    for r in rel_classes:
        print(f"- {r}.")
        
    return rel_classes

In [7]:
def obj_id(o):
    """
    Returns the GUID of the given object, if present, else synthetic ClassName:step_id.

    Non-IfcRoot entities (e.g. IfcMaterial, IfcClassificationReference)
    do not have a GlobalId, so the fallback ensures uniqueness.
    """
    return getattr(o, "GlobalId", f"{o.is_a()}:{o.id()}")

In [8]:
def as_list(v):
    """
    Ensures the value is returned as a list.
    """
    if v is None:
        return []
    return v if isinstance(v, (list, tuple)) else [v]

In [9]:
def edges_maker(model, rel_map=REL_MAP, pset_qto=True):
    """
    For a given model, creates a dictionary of edges (IfcRelationships exploded into from -> to pairs.
    Returns:
        - edges: a dictionary of edges with keys:
            - id, labels, rel_class, rel_id, from, to.
    """

    edges = dict()
    pairs = set()

    for rel_class, (relating_attr, related_attr, label) in rel_map.items():
        for rel in model.by_type(rel_class):
            relating = getattr(rel, relating_attr, None)
            related = getattr(rel, related_attr, None)
            if relating is None:
                continue
            
            # distinction property set vs quantity set:
            if rel_class == "IfcRelDefinesByProperties":
                if pset_qto and relating.is_a("IfcElementQuantity"):
                    edge_label = "DEFINED_BY_QUANTITIES"
                else:
                    edge_label = "DEFINED_BY_PROPERTIES"
            else:
                edge_label = label

            frm = obj_id(relating)
            rel_gid = obj_id(rel)

            # one-to-many relationships:
            for r in as_list(related):
                if r is None:
                    continue
                to  = obj_id(r)
                
                # sanity check:
                pairs.add((rel_gid, frm, to))
                
                edge_id = f"{rel_gid}:{frm}->{to}"
                edges[edge_id]={
                    "id": edge_id,
                    "labels": edge_label,
                    "rel_class": rel_class,
                    "rel_id": rel_gid,
                    "from": frm,
                    "to": to
                }

    expected = len(pairs)
    actual = len(edges)
    
    # print(f"Expected number of edges: {expected}.")
    # print(f"Number of edges: {actual}.")
    
    return expected, actual, edges

## 3.3. IfcEntities - Nodes:

In [10]:
def get_entity_from_id(model, oid):
    """
    Returns the Ifc entity from the given id or GUID.
    """

    if isinstance(oid, str) and (len(oid) == 22) and (":" not in oid):
        return model.by_guid(oid)

    elif ":" in oid:
        try:
            step_id = int(oid.split(":")[-1])
            return model.by_id(step_id)
        except Exception:
            return None

    return None

In [11]:
def get_entities_from_edges(model, edges, typ):
    """
    Returns the relating or related Ifc entities from a dict of edges.
    """

    entities_from_edges = []
    
    for edge in edges.values():
        obj = get_entity_from_id(model, edge[typ])
        if obj:
            entities_from_edges.append(obj) 

    return entities_from_edges

In [12]:
def merge_unique(obj1, obj2):
    """
    Returns a list of unique Ifc entities from two lists of Ifc entities.
    """
    merged = dict()
    for o in obj1 + obj2:
        if o:
            merged[obj_id(o)] = o
    return list(merged.values())

In [13]:
def entities_classes_identify(all_objects):
    """
    Identifies the Ifc entity classes present in the model.
    All IfcElements subtypes collapse to 'IfcElement'.
    All *Type classes collapse to 'IfcTypeObject'.
    """

    clss = set()
    
    for o in all_objects:
        if o.is_a("IfcElement"):
            clss.add("IfcElement")
        elif "Type" in o.is_a():
            clss.add("IfcTypeObject")
        else:
            clss.add(o.is_a())

    return clss

In [14]:
def props_identify(all_objects):

    props = {}

    pset = "IfcPropertySet"
    qset = "IfcElementQuantity"

    for o in all_objects:
        if o.is_a(pset):
            props.setdefault(pset, dict())
            for p in o.HasProperties:
                props[pset][p.Name] = {
                    "property_type": p.is_a()
                }
                
        elif o.is_a(qset):
            props.setdefault(qset, dict())
            for q in o.Quantities:
                props[qset][q.Name] = {
                    "property_type": q.is_a()
                }

    # only IfcMaterial class, no specific material properties to extract beyond the material name.
                
    return props

In [15]:
def pset_props_collect(o):
    """
    Extracts the properties from an IfcPropertySet entity. 
    """

    def unwrap(v):
        """
        Converts an Ifc value wrapper to a plain Python type.
        """
        if v is None:
            return None
        else:
            return getattr(v, "wrappedValue", v)

    props = {}
    
    for p in o.HasProperties:
        if p.is_a("IfcPropertySingleValue"):
            nv = getattr(p, "NominalValue", None)
            props[p.Name] = {
                "kind": "SingleValue",
                "value": unwrap(nv)
            }
        elif p.is_a("IfcPropertyEnumeratedValue"):
            vals = [unwrap(v) for v in (getattr(p, "EnumerationValues", []) or [])]
            enum_ref = getattr(p, "EnumerationReference", None)
            domain = (
                [unwrap(v) for v in (getattr(enum_ref, "EnumerationValues", []) or [])]
                if enum_ref else None
            )
            props[p.Name] = {
                "kind": "EnumeratedValue",
                "value": vals if len(vals) != 1 else vals[0],
                "enumeration_name": getattr(enum_ref, "Name", None) if enum_ref else None,
                "domain": domain
            }

    # if needed, to be developed to handle other kinds of properties.
        
    return props

In [16]:
def qset_props_collect(o, include_none=False):
    """
    Extracts the quantitiy properties from an IfcElementQuantity. 
    """

    props = {}
    
    for q in getattr(o, "Quantities", []) or []:
        val = None
        if q.is_a("IfcQuantityLength"):
            val = q.LengthValue
        elif q.is_a("IfcQuantityArea"):
            val = q.AreaValue
        elif q.is_a("IfcQuantityVolume"):
            val = q.VolumeValue
        elif q.is_a("IfcQuantityCount"):
            val = q.CountValue
        elif q.is_a("IfcQuantityWeight"):
            val = q.WeightValue

        if include_none or val is not None:  
            props[q.Name] = {
                "kind": q.is_a(),
                "value": val
            }

    return props

In [17]:
def node_set(o):
    """
    Extract the node information for a single Ifc entity.
    """

    node = dict()

    # helpers:
    def g_n(o, name, default=None):
        """
        Extract an attribute for an Ifc entity.
        """
        return getattr(o, name, default)

    def basic_props(label, addition=None):
        """
        Set the base node dictionary for an Ifc entity and add it to "node".
        """
        node_id = obj_id(o)
        base = {
            "id": node_id,
            "labels": [label, o.is_a()],
            "name": g_n(o, "Name")
        }
        if addition:
            base.update(addition)
        node[node_id] = base

    
    cls = o.is_a()

    if cls == "IfcProject":
        basic_props("PROJECT")
        
    elif o.is_a("IfcElement"):
        # ObjectType fallback:
        basic_props("ELEMENT", {
            "predefined_type": g_n(o, "PredefinedType", g_n(o, "ObjectType"))
        })
        
    elif cls == "IfcPropertySet":
        basic_props("PSET", {
            "properties": pset_props_collect(o)
        })
        
    elif cls == "IfcElementQuantity":
        basic_props("QSET", {
            "quantities": qset_props_collect(o)
        })
        
    elif "Type" in cls:
        basic_props("TYPE")
        
    elif cls == "IfcMaterial":
        basic_props("MATERIAL")
        
    elif cls in ("IfcSite", "IfcBuilding", "IfcBuildingStorey", "IfcSpace"):
        basic_props("SPATIAL")
        
    elif cls == "IfcClassificationReference":
        scheme = g_n(o, "ReferencedSource") # IfcClassification
        basic_props("CLASSIFICATION_REF",{
            "code": g_n(o, "Identification"),
            "uri": g_n(o, "Location"),
            # link to IfcClassification by id:
            "scheme_id": obj_id(scheme) if scheme else None,
        })

    return node        

In [18]:
def nodes_maker(all_objects):
    """
    For a given model, creates a dictionary of nodes from a list of Ifc entities.
    """

    nodes = {}

    for o in all_objects:
        nodes.update(node_set(o))

    return nodes

In [19]:
def flatten_dict(d, parent_key="", sep="_"):
    """
    Recursively flatten a nested dictionary.
    """
    items = {}
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.update(flatten_dict(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

In [20]:
def nodes_flatten(nodes):
    """
    Flatten nested dictionaries in node attributes for Neo4J comptability.
    """

    def flatten_dict(d, parent_key="", sep="::"):
        """
        Recursively flatten a nested dictionary.
        """
        items = {}
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                items.update(flatten_dict(v, new_key, sep=sep))
            else:
                items[new_key] = v
        return items

    nodes_ready = dict()
    
    for node in nodes.values():
        # preserve "labels"
        base = {k: v for k, v in node.items() if k != "labels"}
        flat = flatten_dict(base)
        flat["labels"] = node["labels"]
        nodes_ready[node["id"]] = flat

    return nodes_ready

## 3.4. Sanity Checks:  

In [21]:
def elements_inspection(model, ifc_type="IfcProduct"):
    """
    Counts and lists IfcElements (excluding proxies in this case) grouped by class.
    Returns:
        - elements: list.
        - classes_counter: Counter.
    """
    
    products = model.by_type(ifc_type)
    elements = [p for p in products if (p.is_a("IfcElement")) and not (p.is_a("IfcBuildingElementProxy"))]
    elements_classes = [el.is_a() for el in elements]
    classes_counter = Counter(elements_classes)

    print(f"Total number of elements: {len(elements)}")
    for cl, count in classes_counter.items():
        print(f"{cl}: {count}.")

    return classes_counter

In [22]:
def sanity(expected, actual, relating, related):
    """
    Sanity checks:
        - Proxies presence.
        - Edge count.
        - Relating entities count.
        - Related entities count.
        - Union nodes.
    Returns:
        bool: True if all checks pass, else False.
    """
    passed = True

    proxies = []
    
    for e in all_ents:
        if "Proxy" in e.is_a():
            proxies.append(e.id())

    if len(proxies) == 0:
        print("Proxy check passed, there is no Proxy entity.")
    else:
        print(f"ERROR: Presence of Proxy entities: {proxies}")
        passed = False 
            
    if expected == actual:
        print(f"Edge count check passed: expected and actual are consistent ({actual}).")
    else:
        print(f"ERROR: edges count mismatch - expected {expected} but got {actual}.")
        passed = False
        
    if len(relating) == actual:
        print(f"Relating entities count check passed: {len(relating)} = number of edges ({actual}).")
    else:
        print(f"ERROR: relating entities count mismatch - edges {actual}, relating entities {len(relating)}.")
        passed = False
        
    if len(related) == actual:
        print(f"Related entities count check passed: {len(related)} = number of edges ({actual}).")
    else:
        print(f"ERROR: related entities count mismatch - edges {actual}, related entities {len(related)}.")
        passed = False

    relating_set = set(relating)
    related_set = set(related)
    union_nodes = relating_set | related_set

    if len(union_nodes) >= max(len(relating_set), len(related_set)):
        print(f"Unique node union check passed: |union|={len(union_nodes)}, |relating|={len(relating_set)}, |related|={len(related_set)}")
    else:
        print(f"ERROR: union size invalind - |union|={len(union_nodes)}, |relating|={len(relating_set)}, |related|={len(related_set)}")

    print(f"The overall sanity check has been succesfully passed: {passed}.")

## 3.5. Graph schema definition:

In [23]:
def ent_labels(ent_type, input, verbose=False):
    """
    Returns:
    - if ent_type == "nodes": (labels, categories, types).
    - if ent_type == "edges": edges types.
    """
    if ent_type == "edges":
        
        input_types = set()
        type_counter = Counter()
    
        for i in input.values():
            label = i["labels"]
            input_types.add(label)
            type_counter[label] += 1 
            
        print(f"Number of edge types: {len(input_types)}.")

        if verbose:
                print()
                print(f"TOTAL NUMBER OF NODES: {len(input)}")
                print()
                print("EDGE TYPES:")
                for t in sorted(type_counter):
                    print(f" - {t}: {type_counter[t]}.")
            
        return type_counter
    
    elif ent_type == "nodes":
        
        input_labels = set()
        input_cats = set()
        input_types = set()
        cat_counter = Counter()
        type_counter = Counter()
        
        for i in input.values():
            labels = i["labels"]
            cat, typ = labels
            input_labels.add((cat, typ))
            input_cats.add(cat)
            input_types.add(typ)
            cat_counter[cat] += 1
            type_counter[typ] += 1
        
        print(f"Number of node categories: {len(input_cats)}.")
        print(f"Number of node types: {len(input_types)}.")
    
        if verbose:
            print()
            print(f"TOTAL NUMBER OF NODES: {len(input)}")
            print()
            print("NODE CATEGORIES:")
            for c in sorted(cat_counter):
                print(f" - {c}: {cat_counter[c]}.")
            print()
            print("NODE TYPES:")
            for t in sorted(type_counter):
                print(f" - {t}: {type_counter[t]}.")
        
        return cat_counter, type_counter

In [24]:
def search_attr(ents, verbose=False, entity=None):
    """
    Analyse the node or edge attributes by category and collect the schema profiles.
    """

    vals = ents.values()
    
    # CATEGORY: top level attributes per category
    cat_attr = defaultdict(set)
    cat_attr_counts = defaultdict(Counter)
    cat_totals = Counter()

    if entity == "edge":
        skip_keys = {"labels", "from", "to"}
    else:
        skip_keys = {"labels"}
    
    for v in vals:
        if entity == "node":
            cat = v["labels"][0]
        else:
            cat = v["labels"]
        cat_totals[cat] += 1
        for a in v.keys():
            if a in skip_keys:
                continue
            cat_attr[cat].add(a)
            cat_attr_counts[cat][a] += 1

    if entity == "node":

        # helpers for flattened dictionaries:
        def prefixed_keys(d, prefix="properties", sep="::"):
            """
            Yield flattened keys in the given dictionary.
            """
            for k in d.keys():
                if k==prefix or k.startswith(prefix + sep):
                    yield k
                        
        # PSET: property names per PSET
        pset_prop = defaultdict(set)
        pset_prop_counts = defaultdict(Counter)
        pset_missing = []
        
        for v in vals:
            if v["labels"][0] != "PSET":
                continue
            name = v.get("name")
            keys = list(prefixed_keys(v))
            if "id" in v:
                keys.append("id")
            if "name" in v:
                keys.append("name")
                
            if not keys:
                pset_missing.append(name)
            for k in keys:
                pset_prop[name].add(k)
                pset_prop_counts[name][k] += 1
        
        # QSET: quantity names per Qset
        qset_prop = defaultdict(set)
        qset_prop_counts = defaultdict(Counter)
        qset_missing = []
        
        for v in vals:
            if v["labels"][0] != "QSET":
                continue
            name = v.get("name")
            keys = list(prefixed_keys(v, prefix="quantities"))
            if "id" in v:
                keys.append("id")
            if "name" in v:
                keys.append("name")
            
            if not keys:
                qset_missing.append(name)
            for k in keys:
                qset_prop[name].add(k)
                qset_prop_counts[name][k] += 1

    if not verbose:

        print(f"\n==={entity.upper()} ATTRIBUTE ANALYSIS===\n")
        print("Top-level attributes per category:")
        print(cat_attr)
        print(cat_attr_counts)
        print(cat_totals)
        print()
        
        if entity == "node":
            
            print("Property names per Pset:")
            print(pset_prop)
            if pset_missing:
                print("Missing PSET keys for:", pset_missing)
            print()
            print("Quantities names per Qset:")
            print(qset_prop)
            print(qset_prop_counts)
            if qset_missing:   # NEW
                print("Missing QSET keys for:", qset_missing)

    if verbose:

        print(f"\n==={entity.upper()} ATTRIBUTE ANALYSIS===\n")
        print("Top-level attributes per category:")
        for cat in sorted(cat_attr):
            print(f" - {cat} ({cat_totals[cat]} nodes): {sorted(cat_attr[cat])}")
            for a, c in cat_attr_counts[cat].most_common():
                print(f"    • {a}: {c}/{cat_totals[cat]}")
            print()

        if entity == "node":
    
            print("\n PSET property keys per set:")
            for pset in sorted(pset_prop):
                print(f" - {pset}: {sorted(pset_prop[pset])}")
                for k, c in pset_prop_counts[pset].most_common():
                    print(f"    • {k}: {c}")
                print()
            if pset_missing:
                print("Missing PSET keys for:", pset_missing)
        
            print("\n QSET quantity keys per set:")
            for qset in sorted(qset_prop):
                print(f" - {qset}: {sorted(qset_prop[qset])}")
                for k, c in qset_prop_counts[qset].most_common():
                    print(f"    • {k}: {c}")
                print()
            if qset_missing:
                print("Missing QSET keys for:", qset_missing)

    if entity == "node":
    
        return (
            cat_attr_counts, cat_totals, 
            pset_prop_counts, 
            qset_prop_counts
        )

    if entity == "edge":

        return (
            cat_attr_counts, cat_totals
        )

## 3.6. Nodes insertion & Relationshsips/edges insertion:

In [25]:
def flatten_dict(dico, parent_key = "", sep="_"):
    """Flatten a nested dictionary recursively."""

    flattened = dict()

    for k, v in dico.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            flattened.update(flatten_dict(v, new_key, sep=sep))
        else:
            flattened[new_key] = v

    return flattened

In [26]:
def driver_merge(ents, entity=None):
    """
    Merge nodes or edges into Neo4J from prepared dictionaries.
    """

    with driver.session(database=DATABASE) as session:

        if entity == "node":

            for node in ents.values():
                labels = ":"+":".join(node["labels"])
                
                def _normalize(v):
                    """
                    In Neo4J, attributes/properties cannot be null,
                    """
                    return "MISSING" if v is None else v

                attr = {k: _normalize(v) for k, v in node.items() if k not in ("labels", "id")}
        
                cypher_node = f"""
                MERGE (n{labels}{{id: $id}})
                SET n += $attr
                """
        
                session.run(cypher_node, {"id": node["id"], "attr": attr}) 

        elif entity == "edge":

            for edge in ents.values():
                attr = {k: v for k, v in edge.items() if k not in ("from", "to", "labels", "rel_id")}
        
                cypher_edge = f"""
                MATCH (a {{id: $from_id }}), (b {{id: $to_id}})
                MERGE (a)-[r:{edge["labels"]}{{rel_id: $rel_id}}]->(b)
                SET r += $attr
                """
        
                session.run(cypher_edge, {
                    "from_id": edge["from"],
                    "to_id": edge["to"],
                    "rel_id": edge["rel_id"],
                    "attr": attr
                        })  

## 3.7. Graph integrity:

In [27]:
def check_labels(expected, dico, entities=None, label_kind=None):
    """
    Compare the expected vs the actual counts in Neo4J:
        - Node categories/types (if entities == "nodes")
        - Edge types (if entities == "edges")
    """

    exp_k = list(expected.keys())

    if entities == "nodes":
        cypher = """
        UNWIND $exp_k AS k
        MATCH (n)
        WHERE k IN labels(n)
        RETURN k AS item, count(*) AS c
        """
        
    elif entities == "edges":
        cypher = """
        UNWIND $exp_k AS k
        MATCH ()-[r]->()
        WHERE type(r) = k
        RETURN k AS item, count(*) As c
        """
        
    else:
        raise ValueError("entities must be 'nodes' or 'edges'.")
    
    with driver.session(database=DATABASE) as session:
        results = session.run(cypher, {"exp_k": exp_k}).data()

    actual = { r["item"]: r["c"] for r in results}
    
    # check:
    all_keys = sorted(set(expected) | set(actual))
    passed = True
    
    for a in all_keys:
        exp = expected.get(a, 0)
        act = actual.get(a, 0)
        if exp != act:
            print(f"{a}, expected = {exp}, actual = {act}, difference = {abs(exp - act)}.")
            passed = False
            
    print(f"The {entities} {label_kind} match: {passed}.")

    dico[f"{entities}_{label_kind}"] = passed

    return dico

In [28]:
def compare_counters_ifc_neo(ifc, neo, dico, key=None):
    """
    Compare defaultdict(Counter) from IFC parsing vs Neo4J extraction.
    """

    diffs = []
    labels = set(ifc) | set(neo)
    
    for label in sorted(labels):
        a = ifc.get(label, Counter())
        b = neo.get(label, Counter())
        keys = set(a.keys()) | set(b.keys())
        for k in sorted(keys):
            x = a.get(k, 0)
            y = b.get(k, 0)
            if x != y:
                diffs.append((label, k, x, y))

    passed = (len(diffs) == 0)
# 
    if passed:
        print("The IFC and the Neo4J defaultdict(Counter) match: True.")

    else:
        for label, k, x, y in diffs:
            print(f"[{label}] key {k}: IFC={x}, NEO={y}")
        print(f"The IFC and the Neo4J defaultdict(Counter) match: False ({len(diffs)} differences).")

    dico[key] = passed

    return dico

In [29]:
def graph_integrity_conclusion(graph):
    """
    Print and evaluate the overall grapch integrity tests.
    """

    results = []

    for key, value in graph.items():
        label = " ".join(key.split("_")).capitalize()
        print(f"{label} integrity test passed: {value}.")
        results.append(value)

    if all(results):
        print()
        print("The graph integrity is assured.")
        return True

    else:
        print()
        print("The graph integrity CANNOT be assured.")
        return False 

# 4. Data load:

In [30]:
model = ifcopenshell.open(IFC_PATH)

In [31]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# 5. Proxy entities handling:

## Proxy Cleanup Strategy

Proxy elements are removed at the start of the workflow to maintain a clean and semantically consistent graph.

1. **Dropping proxies**  
   In this dataset, proxies represent geolocation artifacts or placeholders. They provide no analytical value but increase noise and graph size.

2. **Early removal**  
   Performing the cleanup at the beginning simplifies the workflow by eliminating the need for repeated exclusions later and by preventing dangling references in subsequent steps.

3. **Related relationships and dependent entities**  
   Proxies are often connected through relationships (e.g., `IfcRelDefines*`, `IfcRelAssociates*`) and may reference entities that exist only through those links. Removing both the relationships and such dependent entities avoids the presence of isolated nodes.

4. **Deletion order**  
   - Relationships connected to proxies are deleted first to ensure a clean break.  
   - Dependent isolated entities are removed next, as they then have degree zero.  
   - Finally, the proxies themselves are removed.  

This sequence ensures a compact, consistent model without dangling edges or orphaned entities.

In [32]:
model = proxies_drop(model)


====PROXIES REMOVAL COUNTS:====

relationships: 8
isolated nodes: 3
proxies: 5


# 6. IFC Parsing:

The parsing process begins with the IfcRelationships, from which a dictionary of Edges is created to support Neo4j graph construction. Each edge captures the "from" and "to" attributes that define the connected IFC entities. Based on these connections, all distinct entities can be identified and represented in a dictionary of Nodes. Inverse relationships are not included, as Neo4j inherently supports bidirectional traversal of edges; storing them explicitly would only duplicate information and increase memory usage. Proxy entities are removed in this model, as they are not significant building elements but instead geolocation artifacts.   

To ensure consistency, node identifiers are normalized (e.g. through unique obj_id values), and key metadata such as IFC class labels and relevant properties are attached to nodes and edges. This guarantees that the resulting graph remains both semantically clear and efficient for querying.  

## 6.1. IfcRelationships - Edges Preparation:

### 6.1.1. Functions Building Guidance:

**edges_maker:**

In [33]:
identify_rel(model)

Identified relationship classes:
- IfcRelContainedInSpatialStructure.
- IfcRelDefinesByType.
- IfcRelAggregates.
- IfcRelAssociatesMaterial.
- IfcRelDefinesByProperties.
- IfcRelAssociatesClassification.


{'IfcRelAggregates',
 'IfcRelAssociatesClassification',
 'IfcRelAssociatesMaterial',
 'IfcRelContainedInSpatialStructure',
 'IfcRelDefinesByProperties',
 'IfcRelDefinesByType'}

### 6.1.2. Edges Dictionary:

In [34]:
expected, actual, edges = edges_maker(model)

In [35]:
# edges

## 6.2. IfcEntities - Nodes Preparation:

### 6.2.1. Collect IfcEntities:

In [36]:
relating_ents = get_entities_from_edges(model, edges, "from")

In [37]:
related_ents = get_entities_from_edges(model, edges, "to")

In [38]:
all_ents = merge_unique(relating_ents, related_ents)

### 6.2.2. Functions Building Guidance:

**node_set:**

In [39]:
entities_classes_identify(all_ents)

{'IfcBuilding',
 'IfcBuildingStorey',
 'IfcClassificationReference',
 'IfcElement',
 'IfcElementQuantity',
 'IfcMaterial',
 'IfcProject',
 'IfcPropertySet',
 'IfcSite',
 'IfcTypeObject'}

**pset_props_collect & qset_props_collect:**

In [40]:
props = props_identify(all_ents)
props

{'IfcPropertySet': {'ConstructionMethod': {'property_type': 'IfcPropertySingleValue'},
  'Status': {'property_type': 'IfcPropertyEnumeratedValue'},
  'IsExternal': {'property_type': 'IfcPropertySingleValue'},
  'LoadBearing': {'property_type': 'IfcPropertySingleValue'}},
 'IfcElementQuantity': {'NetVolume': {'property_type': 'IfcQuantityVolume'},
  'Width': {'property_type': 'IfcQuantityLength'},
  'Length': {'property_type': 'IfcQuantityLength'},
  'NetSideArea': {'property_type': 'IfcQuantityArea'},
  'CrossSectionArea': {'property_type': 'IfcQuantityArea'}}}

### 6.2.3. Nodes dictionary:

In [41]:
nodes = nodes_maker(all_ents)

### 6.2.4. Nodes dictionary flattening:

### Flattening the Nodes Dictionary

Neo4j node properties must be stored as **primitive types** (strings, numbers, booleans, lists).  
Nested dictionaries or maps are **not supported** as property values.  

The IFC parsing stage produces a `nodes` dictionary where attributes such as property sets or quantities may be stored as nested structures. To make these compatible with Neo4j:

- Each nested dictionary is **flattened** into key–value pairs (e.g., `{"Pset_WallCommon": {"FireRating": "2h"}}` becomes `{"Pset_WallCommon_FireRating": "2h"}`).
- The `"labels"` field is preserved separately to define the node categories in Neo4j.

This flattening step ensures that all node attributes can be safely inserted as Neo4j properties without loss of information or structural conflicts.

In [42]:
nodes = nodes_flatten(nodes)

## 6.3. Sanity Checks:

In [43]:
elements_distribution = elements_inspection(model)
# elements_distribution

Total number of elements: 15
IfcBeam: 6.
IfcChimney: 1.
IfcFooting: 1.
IfcRoof: 1.
IfcWall: 4.
IfcDiscreteAccessory: 2.


In [44]:
sanity(expected, actual, relating_ents, related_ents)

Proxy check passed, there is no Proxy entity.
Edge count check passed: expected and actual are consistent (65).
Relating entities count check passed: 65 = number of edges (65).
Related entities count check passed: 65 = number of edges (65).
Unique node union check passed: |union|=56, |relating|=41, |related|=19
The overall sanity check has been succesfully passed: True.


# 7. Neo4J graph:

## 7.1. Graph schema definition:

By collecting node labels, categories, types, edge types, and the attributes and properties of both nodes and edges during IFC parsing, a baseline schema profile of the model is established. Once the data is imported into Neo4j, the same metrics can be recomputed with Cypher and compared against this baseline. This provides a sanity check to confirm that the Neo4j graph faithfully reflects the IFC structure, preserves attributes and relationships, and has not lost or altered information during transformation. In this way, the metrics act as a fingerprint of the IFC dataset, supporting the integrity and consistency of the final graph.  

### 7.1.1. Nodes labels, categories and types:

In [45]:
n_cat_counter, n_type_counter = ent_labels("nodes", nodes, True)

Number of node categories: 8.
Number of node types: 20.

TOTAL NUMBER OF NODES: 56

NODE CATEGORIES:
 - CLASSIFICATION_REF: 1.
 - ELEMENT: 15.
 - MATERIAL: 4.
 - PROJECT: 1.
 - PSET: 12.
 - QSET: 10.
 - SPATIAL: 4.
 - TYPE: 9.

NODE TYPES:
 - IfcBeam: 6.
 - IfcBeamType: 1.
 - IfcBuilding: 1.
 - IfcBuildingStorey: 1.
 - IfcChimney: 1.
 - IfcChimneyType: 1.
 - IfcClassificationReference: 1.
 - IfcDiscreteAccessory: 2.
 - IfcDiscreteAccessoryType: 1.
 - IfcElementQuantity: 10.
 - IfcFooting: 1.
 - IfcFootingType: 1.
 - IfcMaterial: 4.
 - IfcProject: 1.
 - IfcPropertySet: 12.
 - IfcRoof: 1.
 - IfcRoofType: 1.
 - IfcSite: 2.
 - IfcWall: 4.
 - IfcWallType: 4.


### 7.1.2. Relationships/edges types:

In [46]:
e_type_counter = ent_labels("edges", edges, True)

Number of edge types: 7.

TOTAL NUMBER OF NODES: 65

EDGE TYPES:
 - AGGREGATES: 12.
 - ASSOCIATED_CLASSIFICATION: 1.
 - ASSOCIATED_MATERIAL: 14.
 - CONTAINED_IN: 1.
 - DEFINED_BY_PROPERTIES: 12.
 - DEFINED_BY_QUANTITIES: 10.
 - DEFINED_BY_TYPE: 15.


### 7.1.3. Nodes & edges attributes:

**NODES ATTRIBUTES & PROPERTIES:**

In [47]:
n_cat_attr_counts, n_cat_totals, n_pset_prop_counts, n_qset_prop_counts = search_attr(nodes, True, "node")


===NODE ATTRIBUTE ANALYSIS===

Top-level attributes per category:
 - CLASSIFICATION_REF (1 nodes): ['code', 'id', 'name', 'scheme_id', 'uri']
    • id: 1/1
    • name: 1/1
    • code: 1/1
    • uri: 1/1
    • scheme_id: 1/1

 - ELEMENT (15 nodes): ['id', 'name', 'predefined_type']
    • id: 15/15
    • name: 15/15
    • predefined_type: 15/15

 - MATERIAL (4 nodes): ['id', 'name']
    • id: 4/4
    • name: 4/4

 - PROJECT (1 nodes): ['id', 'name']
    • id: 1/1
    • name: 1/1

 - PSET (12 nodes): ['id', 'name', 'properties::ConstructionMethod::kind', 'properties::ConstructionMethod::value', 'properties::IsExternal::kind', 'properties::IsExternal::value', 'properties::LoadBearing::kind', 'properties::LoadBearing::value', 'properties::Status::domain', 'properties::Status::enumeration_name', 'properties::Status::kind', 'properties::Status::value']
    • id: 12/12
    • name: 12/12
    • properties::Status::kind: 11/12
    • properties::Status::value: 11/12
    • properties::Status::enum

**EDGES ATTRIBUTES:**

In [48]:
e_cat_attr_counts, e_cat_totals = search_attr(edges, True, "edge")


===EDGE ATTRIBUTE ANALYSIS===

Top-level attributes per category:
 - AGGREGATES (12 nodes): ['id', 'rel_class', 'rel_id']
    • id: 12/12
    • rel_class: 12/12
    • rel_id: 12/12

 - ASSOCIATED_CLASSIFICATION (1 nodes): ['id', 'rel_class', 'rel_id']
    • id: 1/1
    • rel_class: 1/1
    • rel_id: 1/1

 - ASSOCIATED_MATERIAL (14 nodes): ['id', 'rel_class', 'rel_id']
    • id: 14/14
    • rel_class: 14/14
    • rel_id: 14/14

 - CONTAINED_IN (1 nodes): ['id', 'rel_class', 'rel_id']
    • id: 1/1
    • rel_class: 1/1
    • rel_id: 1/1

 - DEFINED_BY_PROPERTIES (12 nodes): ['id', 'rel_class', 'rel_id']
    • id: 12/12
    • rel_class: 12/12
    • rel_id: 12/12

 - DEFINED_BY_QUANTITIES (10 nodes): ['id', 'rel_class', 'rel_id']
    • id: 10/10
    • rel_class: 10/10
    • rel_id: 10/10

 - DEFINED_BY_TYPE (15 nodes): ['id', 'rel_class', 'rel_id']
    • id: 15/15
    • rel_class: 15/15
    • rel_id: 15/15



## 7.2. Graph creation:

To set up the Neo4j graph, both the **"nodes"** dictionary and the **"edges"** dictionary generated during the IFC parsing stage will serve as the input. These dictionaries represent the essential building blocks of the graph:  
- the **"nodes"** (with their labels, categories, types, attributes and properties/quantities)  
- the **"edges"** (with their relationship categories and endpoints)

They will be passed into Neo4j through the **Neo4j Python driver**, which manages the connection and Cypher queries used to create the graph.

### 7.2.1. Nodes insertion:

In [49]:
driver_merge(nodes, "node")

### 7.2.2. Relationships/edges insertion:

In [50]:
driver_merge(edges, "edge")

## 7.3. Graph integrity:

In [51]:
graph_integrity = dict()

### 7.3.1. Nodes labels, categories and types:

In [52]:
graph_integrity = check_labels(n_cat_counter, graph_integrity, "nodes", "categories")

The nodes categories match: True.


In [53]:
graph_integrity = check_labels(n_type_counter, graph_integrity, "nodes", "types")

The nodes types match: True.


### 7.3.2. Edges labels:

In [54]:
graph_integrity = check_labels(e_type_counter, graph_integrity, "edges", "types")

The edges types match: True.


### 7.3.3. Nodes attributes and properties:

**PER CATEGORY:**

In [55]:
neo_cat_attr_counts = defaultdict(Counter)

with driver.session(database=DATABASE) as session:
    results = session.run(
        """
        MATCH (n)
        UNWIND keys(n) AS k
        RETURN head(labels(n)) AS label, k, count(*) AS c
        ORDER BY label, k
        """
    )
    for r in results:
        label = r["label"]
        key = r["k"]
        c = r["c"]
        neo_cat_attr_counts[label][key] = c

In [56]:
graph_integrity = compare_counters_ifc_neo(n_cat_attr_counts, neo_cat_attr_counts, graph_integrity, "nodes_categories_attributes")

The IFC and the Neo4J defaultdict(Counter) match: True.


**PER PSET:**

In [57]:
neo_pset_prop_counts = defaultdict(Counter)

with driver.session(database=DATABASE) as session:

    results = session.run(
        """
        MATCH (n:PSET)
        WHERE n.name IS NOT NULL
        UNWIND keys(n) AS k
        RETURN n.name AS pname, k, count(*) AS c
        ORDER BY pname, k
        """
    )

    for r in results:
        pname = r["pname"]
        key = r["k"]
        c = r["c"]
        neo_pset_prop_counts[pname][key] = c

In [58]:
graph_integrity = compare_counters_ifc_neo(n_pset_prop_counts, neo_pset_prop_counts, graph_integrity, "PSET_properties")

The IFC and the Neo4J defaultdict(Counter) match: True.


**PER QSET:**

In [59]:
neo_qset_prop_counts = defaultdict(Counter)

with driver.session(database=DATABASE) as session:

    results = session.run(
        """
        MATCH (n:QSET)
        WHERE n.name IS NOT NULL
        UNWIND keys(n) AS k
        RETURN n.name AS qname, k, count(*) AS c
        ORDER BY qname, k
        """
    )

    for r in results:
        qname = r["qname"]
        key = r["k"]
        c = r["c"]
        neo_qset_prop_counts[qname][key] = c

In [60]:
graph_integrity = compare_counters_ifc_neo(n_qset_prop_counts, neo_qset_prop_counts, graph_integrity, "QSET_properties")

The IFC and the Neo4J defaultdict(Counter) match: True.


### 7.3.4. Edges attributes:

In [61]:
neo_e_cat_attr_counts = defaultdict(Counter)

with driver.session(database=DATABASE) as session:
    
    results = session.run(
        """
        MATCH ()-[r]-()
        UNWIND keys(r) AS k
        RETURN type(r) AS rel_type, k, count(DISTINCT r) As c
        ORDER BY rel_type, k
        """
    )

    for r in results:
        rel_type = r["rel_type"]
        key = r["k"]
        c = r["c"]
        neo_e_cat_attr_counts[rel_type][key] = c

In [62]:
graph_integrity = compare_counters_ifc_neo(e_cat_attr_counts, neo_e_cat_attr_counts, graph_integrity, "edges_attributes")

The IFC and the Neo4J defaultdict(Counter) match: True.


### 7.3.5. Graph integrity conclusion:

In [63]:
graph_integrity_final = graph_integrity_conclusion(graph_integrity)

Nodes categories integrity test passed: True.
Nodes types integrity test passed: True.
Edges types integrity test passed: True.
Nodes categories attributes integrity test passed: True.
Pset properties integrity test passed: True.
Qset properties integrity test passed: True.
Edges attributes integrity test passed: True.

The graph integrity is assured.


## 7.4. Exploration & visualization:

**Important:** The visualization cells in this notebook rely on `ipywidgets` (e.g., `Neo4jGraphWidget`). On classic **Jupyter Notebook 7**, widget rendering is unreliable, so the canvas may appear empty. Please open this notebook in **JupyterLab** to ensure the graph renders.

In [64]:
widget = Neo4jGraphWidget(driver)

with driver.session(database=DATABASE) as session:

    widget._session = session                      # point widget to session
    widget.show_cypher(
        """
        MATCH (n)-[r]->(m) 
        RETURN n, r, m
        """
    )

GraphWidget(layout=Layout(height='800px', width='100%'))

## 7.5. Graph summary:

In [65]:
with driver.session(database=DATABASE) as s:
    sections = ["GRAPH COUNTS", "TOKENS", "META"]
    stats = {
        sec: s.run("CALL db.stats.retrieve($sec) YIELD data RETURN data", sec=sec).single()["data"]
        for sec in sections
    }

# Coerce Neo4j temporal/spatial values to strings so JSON can render
stats_clean = json.loads(json.dumps(stats, default=str))
display(JSON(stats_clean))

<IPython.core.display.JSON object>

In [66]:
with driver.session(database="capstone") as s:
    rec = s.run("""
        CALL apoc.meta.stats()
        YIELD nodeCount, relCount, labels, relTypes, propertyKeyCount
        RETURN nodeCount, relCount, labels, relTypes, propertyKeyCount
    """).single()

In [67]:
rec

<Record nodeCount=56 relCount=65 labels={'IfcElementQuantity': 10, 'IfcFooting': 1, 'IfcFootingType': 1, 'PROJECT': 1, 'IfcWallType': 4, 'SPATIAL': 4, 'IfcMaterial': 4, 'QSET': 10, 'CLASSIFICATION_REF': 1, 'IfcBeam': 6, 'TYPE': 9, 'IfcRoofType': 1, 'IfcChimney': 1, 'IfcProject': 1, 'IfcWall': 4, 'IfcChimneyType': 1, 'IfcRoof': 1, 'IfcPropertySet': 12, 'IfcClassificationReference': 1, 'IfcDiscreteAccessory': 2, 'PSET': 12, 'IfcBuildingStorey': 1, 'ELEMENT': 15, 'IfcBuilding': 1, 'IfcDiscreteAccessoryType': 1, 'MATERIAL': 4, 'IfcBeamType': 1, 'IfcSite': 2} relTypes={'(:CLASSIFICATION_REF)-[:ASSOCIATED_CLASSIFICATION]->()': 1, '(:IfcChimneyType)-[:DEFINED_BY_TYPE]->()': 1, '()-[:AGGREGATES]->(:SPATIAL)': 4, '(:PSET)-[:DEFINED_BY_PROPERTIES]->()': 12, '()-[:AGGREGATES]->(:IfcDiscreteAccessory)': 2, '(:IfcBuilding)-[:AGGREGATES]->()': 1, '()-[:AGGREGATES]->(:IfcBuildingStorey)': 1, '(:IfcFootingType)-[:DEFINED_BY_TYPE]->()': 1, '()-[:AGGREGATES]->()': 12, '(:QSET)-[:DEFINED_BY_QUANTITIES]->

In [68]:
driver.close()